In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
import tensorflow as tf
import numpy as np

In [12]:
import glob


drow= glob.glob(r'D:\BIG_PROJECT\face-detection-yolov8\test_face_images\test_cnn\drowsy\*.*')
normal= glob.glob(r'D:\BIG_PROJECT\face-detection-yolov8\test_face_images\test_cnn\non-drowsy\*.*')


# drow= glob.glob(r'C:\Users\homin\Downloads\test\drowsy\*.*')
# normal= glob.glob(r'C:\Users\homin\Downloads\test\non-drowsy\*.*')


# drow= glob.glob(r'D:\BIG_PROJECT\face-detection-yolov8\test_face_images\test_vit\drowsy\*.*')
# normal= glob.glob(r'D:\BIG_PROJECT\face-detection-yolov8\test_face_images\test_vit\non-drowsy\*.*')


data = []
labels = []




for i in normal:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
    #print(image)
    image=np.array(image)
    data.append(image)
    labels.append(1)  
for i in drow:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(0)

data = np.array(data)
labels = np.array(labels)

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from keras.preprocessing import image

c:\Users\homin\anaconda3\envs\py39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [28]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(224, 224),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
data_augmentation.layers[0].adapt(data)

In [29]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


In [16]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [7]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 1000
image_size = 224  # We'll resize input images to this size
patch_size = 16  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [17]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [24]:
def create_vit_classifier():
    inputs = layers.Input(shape=(224,224,3))
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=projection_dim, 
            dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(2)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [5]:
vgg_local = r"E:\HOC_TAP\NCKH\model\VGG\local_vgg_official.h5"
vgg_cluster1 = r"E:\HOC_TAP\NCKH\model\VGG\cluster1_vgg_500.h5"

In [6]:
den_local = r"E:\HOC_TAP\NCKH\model\dense\local2_densenet_500.h5"
den_cluster1 = r"E:\HOC_TAP\NCKH\model\dense\cluster1_densenet_official.h5"
den_cluster2 = r"E:\HOC_TAP\NCKH\model\dense\cluster2_densenet_official.h5"

In [7]:
vit_local = r"E:\HOC_TAP\NCKH\model\ViT\local_vit_500.h5"
vit_cluster1 = r"E:\HOC_TAP\NCKH\model\ViT\cluster1_vit_500.h5"
vit_cluster2 = r"E:\HOC_TAP\NCKH\model\ViT\cluster2_vit_official2.h5"

In [8]:
resnet_local = r"E:\HOC_TAP\NCKH\model\resnet\local_resnet_official.h5"
resnet_cluster1 = r"E:\HOC_TAP\NCKH\model\resnet\cluster1_resnet_official.h5"

In [9]:
lstm_local = r"E:\HOC_TAP\NCKH\model\resnet\local_resnet_official.h5"
lstm_cluster1 = r"E:\HOC_TAP\NCKH\model\LSTM\cluster1_LSTM_official.h5"
lstm_cluster2 = r"E:\HOC_TAP\NCKH\model\LSTM\cluster2_lstm_official.h5"

In [10]:
model = tf.keras.models.load_model(den_local,compile=True)#Load model with DNN:

In [30]:

#Load model with ViT:
model = create_vit_classifier()
model.load_weights(vit_cluster2)

In [ ]:
print(labels)

[1 1 1 ... 0 0 0]


In [13]:
y_hat = model.predict(data)
y_pred = np.argmax(y_hat, axis=-1)
y_test_label= labels.reshape(-1, 1)
# print(y_test_label)
print(y_pred)
# Tính accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(labels, y_pred)
print('Accuracy: %f' % accuracy)
# Tính precision tp / (tp + fp)
precision = precision_score(labels, y_pred, average='macro')
print('Precision: %f' % precision)
# Tính recall: tp / (tp + fn)
recall = recall_score(labels, y_pred, average='macro')
print('Recall: %f' % recall)
# Tính f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(labels, y_pred, average='macro')
print('F1 score: %f' % f1)

7/7 [==============================] - 14s 2s/step
[1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0]
Accuracy: 0.855000
Precision: 0.855320
Recall: 0.855000
F1 score: 0.854967


############################################## MY TEST################

In [ ]:


def val_data_creator(config, batch_size):
    valid_path = r"D:\BIG_PROJECT\face-detection-yolov8\test_face_images\nhut_faces2"
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        valid_path,
        image_size=(224, 224),
        batch_size=batch_size,
        validation_split=None,
        seed=123,
        label_mode='categorical'
    )
    # val_dataset = val_dataset.map(preprocess)  # Apply preprocessing
    return val_dataset

batch_size = 16
result = model.evaluate(val_data_creator(None, batch_size))  # Pass None as config
print(result)

In [ ]:
images= glob.glob(r'D:\BIG_PROJECT\face-detection-yolov8\test_face_images\test_vit\non-drowsy\*.*')
test = []
for i in images:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (224,224))
    #print(image)
    image=np.array(image)
    test.append(image)

test = np.array(test)

result = model.predict(test)
pre_label = np.argmax(result, axis=-1)
print(pre_label)


4/4 [==============================] - 7s 1s/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
